# Intro to Data Science 
## Part VIII. - Solving Kaggle's Job Salary prediction competition

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import random
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error

## I. Read a small sample
Note: download and extract the **`Train_rev1.csv`** data file from [kaggle](https://www.kaggle.com/c/job-salary-prediction/data).

In [ ]:
random.seed(137)
with open("data/Train_rev1.csv", "r") as infile:
    numlines = len(infile.readlines())
skip_index = random.sample(range(1, numlines), numlines - int(numlines/10))

In [ ]:
data = pd.read_csv("data/Train_rev1.csv", index_col='Id', skiprows=skip_index)
data.head()

### Check null values

In [ ]:
data.isnull().sum() / len(data)

In [ ]:
print 'ContractType'
print data.ContractType.unique()
print
print 'ContractTime'
print data.ContractTime.unique()

In [ ]:
print 'ContractType'
print data.ContractType.describe()
print
print 'ContractTime'
print data.ContractTime.describe()

ContractType and ContractTime could be easily inputted, but since the majority of values are the same, we skip it.

### Train-test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.FullDescription, data.SalaryNormalized, test_size=0.25, random_state=137)

df_train, df_test, y_train, y_test = train_test_split(
    data, 
    data.SalaryNormalized,
    test_size=0.25, 
    random_state=137
)

---

## II. Baseline model

In order to put our models to context we create a baseline model which represents "coin toss" decision.

In [ ]:
class Baseline(BaseEstimator, RegressorMixin):
    
    def fit(self, X, y):
        """Calculate mean value for y.
        
        Parameters
        ----------
        X : array-like
            not used
        y : array-like
            target values
        
        Returns
        -------
        self : object
            Returns self.
        """
        self.value = y.mean()
        return self
    
    def predict(self, X):
        """Returns calculated mean for every item in X.
        
        Parameters
        ----------
        X : array-like
            Input matrix

        Returns
        -------
        y : array-like
            "Predicted" target values for X.
        """
        if not hasattr(self, 'value'):
            raise ValueError("Regressor not trained yet. Use fit method first.")
        target_shape = (X.shape[0], )
        return np.ones(target_shape) * self.value

In [ ]:
basepipe = Pipeline(steps=[
    ("base", Baseline())
])

basepipe.fit(X_train, y_train)

In [ ]:
mean_absolute_error(y_test, basepipe.predict(X_test))

This is our baseline prediction error. 

---

## III. First model: 
### Regression model, using only FullDescription

In [ ]:
tfidf = TfidfVectorizer(min_df=0.1, max_df=0.9, stop_words='english')
tfidf.fit(data.FullDescription)
print "Vocabulary size:", len(tfidf.vocabulary_)

In [ ]:
linpipe = Pipeline(steps=[
    ("tfidf", tfidf),
    ("linreg", LinearRegression())
])
linpipe.fit(X_train, y_train)

print mean_absolute_error(y_test, linpipe.predict(X_test))

In [ ]:
ridegpipe = Pipeline(steps=[
    ("tfidf", tfidf),
    ("rideg", Ridge(random_state=137))
])
ridegpipe.fit(X_train, y_train)

print mean_absolute_error(y_test, ridegpipe.predict(X_test))

### Exercise: Hyperparameter optimization

Optimize `min_df`, `max_df` and `rideg__alpha` parameters!

Don't forget to use the *data.FullDescription* and *data.SalaryNormalized* instead of the train-test splits.

## IV: Using multiple features

Q: What if we need to use different transformations for each feature?  
A: ItemSelector

http://scikit-learn.org/stable/auto_examples/hetero_feature_union.html

In [ ]:
class ItemSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [ ]:
titpipe = Pipeline([
    ('selector', ItemSelector('Title')),
    ('count', CountVectorizer())
])

descpipe = Pipeline([
    ('selector', ItemSelector('FullDescription')),
    ('tfidf', TfidfVectorizer(min_df=0.1, max_df=0.9, stop_words='english'))
])

feat = FeatureUnion([
    ('title', titpipe),
    ('desc', descpipe)
])

In [ ]:
ulti_lin_pipe = Pipeline([
    ('union', feat),
    ('linreg', LinearRegression())
])
ulti_lin_pipe.fit(df_train, y_train)

print mean_absolute_error(y_test, ulti_lin_pipe.predict(df_test))

In [ ]:
ulti_rideg_pipe = Pipeline([
    ('union', feat),
    ('rideg', Ridge(random_state=137))
])
ulti_rideg_pipe.fit(df_train, y_train)

print mean_absolute_error(y_test, ulti_rideg_pipe.predict(df_test))

### Exercise II: Add more columns

Following the same process, add every reasonable column with preprocessing to the pipelines and inspect `LinearRegression` and `Ridge` errors.

### Exercise III: Hyperparameter optimization

Find the previously assembled pipeline's optimal parameters. To easily find the correct parameter names use the pipeline's `get_params` method.

In [ ]:
ulti_rideg_pipe_param_names = ulti_rideg_pipe.get_params().keys()

print "ulti_rideg_pipe's parameter names:"
print '-', '\n- '.join(ulti_rideg_pipe_param_names)